<a href="https://colab.research.google.com/github/bsse1027/Stock_Indicators_Calculator/blob/main/Grab_Company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import csv
import array as arr
from pprint import pprint
import pandas as pd

In [92]:
def getCompanyCodeList():
  response = requests.get("https://www.amarstock.com/LatestPrice/34267d8d73dd?fbclid=IwAR0UNljsm-ezbNkKryoHblOkrZNNzdjUGad6lcqQEydQbKuP7TRbZHYOFr4")
  response.raise_for_status()
  if response.status_code == 200:
    todos = json.loads(response.text)
    all_company_list = []
    for x in todos:
      companyId = x['Scrip']
      all_company_list.append(companyId)
  return all_company_list

In [ ]:
getCompanyCodeList()

In [98]:
def getMovingAverage(companyId):
  response2 = requests.get(f"https://www.amarstock.com/data/afe01cd8b512070a/?scrip={companyId}&cycle=Day1&dtFrom=2016-12-15T05%3A02%3A13.318Z&fbclid=IwAR0qZBhgiqSV6L6xTerlCEsXvVwtaLMaQvTqqMfUmjloMfBO2jocwV95DE8")
  if (response2.status_code == 200):  
    todos2 = json.loads(response2.text)
  json_string = json.dumps(todos2)
  df = pd.read_json(json_string)
  # for i in range(len(df)):
  #   if df.loc[i, 'DateEpoch'] == 1483228800000:
  #     print(f"{df.loc[i, 'DateString']}")
  close_p = df[['Close', 'DateString', 'DateEpoch']].copy()
  close_p['SMA12'] = close_p['Close'].rolling(12).mean()
  final_list = close_p[['DateString', 'DateEpoch', 'SMA12']].copy()
  return final_list.to_dict()

In [ ]:
companyId = "BEXIMCO"
getMovingAverage(companyId)

In [106]:
def getMACD(companyId):
  response2 = requests.get(f"https://www.amarstock.com/data/afe01cd8b512070a/?scrip={companyId}&cycle=Day1&dtFrom=2016-12-15T05%3A02%3A13.318Z&fbclid=IwAR0qZBhgiqSV6L6xTerlCEsXvVwtaLMaQvTqqMfUmjloMfBO2jocwV95DE8")
  if (response2.status_code == 200):  
    todos2 = json.loads(response2.text)
  json_string = json.dumps(todos2)
  df = pd.read_json(json_string)
  # for i in range(len(df)):
  #   if df.loc[i, 'DateEpoch'] == 1483228800000:
  #     print(f"{df.loc[i, 'DateString']}")
  close_p = df[['Close', 'DateString', 'DateEpoch']].copy()
  # Get the 26-day EMA of the closing price
  k = close_p['Close'].ewm(span=12, adjust=False).mean()
  # Get the 12-day EMA of the closing price
  d = close_p['Close'].ewm(span=26, adjust=False).mean()
  # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
  macd = k - d
  # Get the 9-Day EMA of the MACD for the Trigger line
  macd_s = macd.ewm(span=9, adjust=False).mean()
  # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
  macd_h = macd - macd_s
  # Add all of our new values for the MACD to the dataframe
  close_p['macd'] = df.index.map(macd)
  close_p['macd_h'] = df.index.map(macd_h)
  close_p['signal'] = df.index.map(macd_s)
  final_macd = close_p[['DateString', 'DateEpoch', 'macd', 'signal']].copy()
  return final_macd.to_dict()

In [ ]:
companyId = "BEXIMCO"
getMACD(companyId)

In [104]:
def getRSI(companyId):
  response2 = requests.get(f"https://www.amarstock.com/data/afe01cd8b512070a/?scrip={companyId}&cycle=Day1&dtFrom=2016-12-15T05%3A02%3A13.318Z&fbclid=IwAR0qZBhgiqSV6L6xTerlCEsXvVwtaLMaQvTqqMfUmjloMfBO2jocwV95DE8")
  if (response2.status_code == 200):  
    todos2 = json.loads(response2.text)
  json_string = json.dumps(todos2)
  df = pd.read_json(json_string)
  # for i in range(len(df)):
  #   if df.loc[i, 'DateEpoch'] == 1483228800000:
  #     print(f"{df.loc[i, 'DateString']}")
  close_p = df[['Close', 'DateString', 'DateEpoch']].copy()
  # Calculate Price Differences
  close_p['diff'] = close_p['Close'].diff(1)
  # Calculate Avg. Gains/Losses
  close_p['gain'] = close_p['diff'].clip(lower=0).round(2)
  close_p['loss'] = close_p['diff'].clip(upper=0).abs().round(2)

  window_length = 14
  # Get initial Averages
  close_p['avg_gain'] = close_p['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
  close_p['avg_loss'] = close_p['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]


  # Get WMS averages
  # Average Gains
  for i, row in enumerate(close_p['avg_gain'].iloc[window_length+1:]):
      close_p['avg_gain'].iloc[i + window_length + 1] =\
          (close_p['avg_gain'].iloc[i + window_length] *
          (window_length - 1) +
          close_p['gain'].iloc[i + window_length + 1])\
          / window_length
  # Average Losses
  for i, row in enumerate(close_p['avg_loss'].iloc[window_length+1:]):
      close_p['avg_loss'].iloc[i + window_length + 1] =\
          (close_p['avg_loss'].iloc[i + window_length] *
          (window_length - 1) +
          close_p['loss'].iloc[i + window_length + 1])\
          / window_length
  # Calculate RS
  close_p['rs'] = close_p['avg_gain'] / close_p['avg_loss']
  # Calculate RSI
  close_p['rsi'] = 100 - (100 / (1.0 + close_p['rs']))
  final_rsi = close_p[['DateString', 'DateEpoch', 'rsi']].copy()
  return final_rsi.to_dict()

In [ ]:
companyId = "BEXIMCO"
getRSI(companyId)